In [1]:
%%time
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys
import math
import os
import glob

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

CPU times: user 2.75 s, sys: 397 ms, total: 3.15 s
Wall time: 3.28 s


In [2]:
%%time
path = '/glade/u/home/noteng/work/research/data/'
file = 'march13-march14.nc'
data = xr.open_dataset(path+file)
data = data.sel(time=slice('2020-03-13T04:00:00.000000000', '2020-03-14T05:00:00.000000000'))
data.close()

CPU times: user 53.3 ms, sys: 13 ms, total: 66.3 ms
Wall time: 137 ms


In [3]:
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,450:580,256:771] #Based on longitude and latitude of Andoya and Norwegian Sea
equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,250:650,450:850] #Based on longitude and latitude of Andoya and Norwegian Sea
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,330:580,660:780] #### hdm1 and hdm2
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor']
equivalent_reflectivity_factor

<xarray.DataArray 'equivalent_reflectivity_factor' (time: 301, Yc: 400, Xc: 400)>
[48160000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
  * Xc       (Xc) float32 -8.3e+04 -8.2e+04 -8.1e+04 ... 3.15e+05 3.16e+05
  * Yc       (Yc) float32 -2.06e+06 -2.061e+06 ... -2.458e+06 -2.459e+06
    lon      (Yc, Xc) float32 ...
    lat      (Yc, Xc) float32 ...
Attributes:
    standard_name:  equivalent_reflectivity_factor
    long_name:      Radar Reflectivity
    units:          dBZ
    grid_mapping:   projection_lambert

In [4]:
# load Mask_ERF_files
Mask_ERF = iris.load('../saved-files/threshold-all/Mask_ERF_iris-all-all.nc')[0]
Mask_ERF

<iris 'Cube' of segmentation_mask / (1) (time: 301; -- : 400; -- : 400)>

In [5]:
track = pd.read_csv('../saved-files/threshold-all/track-reset-all-all.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,399,222.481123,301.518877,2,-5,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.282481e+06,218518.877030,69.351172,15.468683,1,0 days 00:00:00
1,0,418,284.000000,229.000000,1,-5,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.344000e+06,146000.000000,68.844109,13.564160,2,0 days 00:00:00
2,0,423,310.727163,278.000000,2,-5,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.370727e+06,195000.000000,68.568710,14.702179,3,0 days 00:00:00
3,0,444,39.000000,369.000000,1,0,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.099000e+06,286000.000000,70.939285,17.759075,4,0 days 00:00:00
4,1,569,39.000000,372.000000,1,5,370,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.099000e+06,289000.000000,70.935585,17.839457,4,0 days 00:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66394,300,511,190.000000,378.523054,2,15,66349,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.250000e+06,295523.054421,69.566104,17.482600,15319,0 days 00:00:00
66395,300,512,192.713674,297.979195,8,15,66350,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.252714e+06,214979.195406,69.624180,15.451297,15320,0 days 00:00:00
66396,300,540,298.000000,225.000000,1,15,66362,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.358000e+06,142000.000000,68.718887,13.446220,15321,0 days 00:00:00
66397,300,563,215.000000,290.000000,1,20,66376,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.275000e+06,207000.000000,69.428795,15.198971,15322,0 days 00:00:00


### CALCULATING FOR DISTANCE

In [6]:
distance = []
for i in range(len(track)):
    lat_andenes = 69.141599 # latitude of Andenes
    # lat_andenes = 69.2412 # latiude of the radar
    lon_andenes = 15.684104 # latitude of Andenes
    # lon_andenes = 16.0030 # longitude of where the radar is located
    
# 69.2412° N, 16.0030° E
    
    distances = tobac.analysis.haversine(track['latitude'][i], track['longitude'][i], lat_andenes, lon_andenes)
    distance.append(distances)

In [7]:
distance[:10]
# np.max(distance)

[24.82834010089355,
 90.84217589394397,
 74.97516282075013,
 215.0566523817313,
 215.8144067323939,
 125.68033936573997,
 122.97712651988019,
 120.3737662531601,
 119.04064432529773,
 114.85915933637442]

In [8]:
%%time
# apped distance to track dataframe
track['distance'] = pd.Series(distance)

CPU times: user 7.96 ms, sys: 995 µs, total: 8.95 ms
Wall time: 8.96 ms


In [9]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,399,222.481123,301.518877,2,-5,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.282481e+06,218518.87703,69.351172,15.468683,1,0 days 00:00:00,24.828340
1,0,418,284.000000,229.000000,1,-5,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.344000e+06,146000.00000,68.844109,13.564160,2,0 days 00:00:00,90.842176
2,0,423,310.727163,278.000000,2,-5,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.370727e+06,195000.00000,68.568710,14.702179,3,0 days 00:00:00,74.975163
3,0,444,39.000000,369.000000,1,0,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.099000e+06,286000.00000,70.939285,17.759075,4,0 days 00:00:00,215.056652
4,1,569,39.000000,372.000000,1,5,370,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.099000e+06,289000.00000,70.935585,17.839457,4,0 days 00:05:00,215.814407


## _cells that lived for at least 10 mins old_ 

In [10]:
%%time
# cell_one_hour = np.timedelta64( 600, 's') #10 minutes in seconds
cell_times = np.timedelta64( 600, 's') #10 minutes in seconds

#convert time_cell to seconds
time_cell = track['time_cell'].astype('timedelta64[ns]').values

# track cell that live for at least 10 minutes long
# track_close_to_AMF1 = track[ time_cell >= cell_times ]
track_close_to_AMF1 = track[ (time_cell >= cell_times)]
# track_close_to_AMF1 = track[ ((track['distance'] <= 95) & (track['time_cell'].values >= cell_one_hour)) ]
track_close_to_AMF1= track_close_to_AMF1.drop_duplicates(subset=['cell']) # remove duplicates of cell_ids
track_close_to_AMF1.head()

CPU times: user 132 ms, sys: 40 µs, total: 132 ms
Wall time: 132 ms


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
7,2,716,127.000000,331.000000,1,10,780,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.187000e+06,248000.000000,70.187965,16.469553,5,0 days 00:10:00,120.373766
12,2,598,141.532499,278.000000,2,5,734,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.201532e+06,195000.000000,70.104838,15.061745,6,0 days 00:10:00,109.903521
20,2,490,169.940029,290.190262,4,0,688,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.229940e+06,207190.262173,69.837291,15.308279,10,0 days 00:10:00,78.817076
29,2,619,207.917634,251.082366,2,5,743,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.267918e+06,168082.366467,69.522261,14.238618,13,0 days 00:10:00,70.856104
33,2,624,210.000000,289.544393,2,5,745,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.270000e+06,206544.393387,69.474535,15.198953,14,0 days 00:10:00,41.685329


In [11]:
# total track cells found.
len(track_close_to_AMF1) 

7121

# tracking all cells that were at least 10 minutes old back in time to get all the cells

In [12]:
%%time
combined_tracks_AMF1 = []
for index, cell_id in enumerate(np.unique(track_close_to_AMF1['cell'].values)): # looped through tracks found with AMF1 site
    if cell_id in track['cell']: # checking if those cells(tracks) are within the original tracked dataframe
        track_i = track[track['cell'] == cell_id] # display it if it is within the tracked dataframe
        combined_tracks_AMF1.append(track_i) # append the results you get a new dataFrame calld combined_tracks_AMF1

# Concatenate the dataframes(combined_tracks_AMD) along the rows axis with keys
# tracks_closer_AMF1 = pd.concat(combined_tracks_AMF1, keys=['df1', 'df2', 'df3', 'df4', 'df5', 'df6'])
tracks_closer_AMF1 = pd.concat(combined_tracks_AMF1)
tracks_closer_AMF1

CPU times: user 3.52 s, sys: 63.6 ms, total: 3.58 s
Wall time: 3.66 s


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
5,0,453,121.000000,324.705104,2,0,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.181000e+06,241705.104259,70.248601,16.323888,5,0 days 00:00:00,125.680339
6,1,581,124.000000,327.838844,2,5,374,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.184000e+06,244838.843751,70.218311,16.396476,5,0 days 00:05:00,122.977127
7,2,716,127.000000,331.000000,1,10,780,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.187000e+06,248000.000000,70.187965,16.469553,5,0 days 00:10:00,120.373766
8,3,578,128.746166,333.673351,6,5,1063,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.188746e+06,250673.350542,70.169425,16.533521,5,0 days 00:15:00,119.040644
9,4,611,133.000000,334.000000,1,5,1406,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.193000e+06,251000.000000,70.130630,16.529381,5,0 days 00:20:00,114.859159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66339,299,476,293.326516,390.199380,15,15,66254,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.353327e+06,307199.379687,68.618348,17.437243,15279,0 days 00:05:00,91.303475
66340,300,539,295.648140,392.416781,10,15,66361,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.355648e+06,309416.781206,68.594721,17.483020,15279,0 days 00:10:00,94.427981
66343,298,555,260.000000,211.000000,1,25,66190,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.320000e+06,128000.000000,69.072189,13.157945,15281,0 days 00:00:00,100.574806
66344,299,531,260.153314,213.000000,3,25,66288,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.320153e+06,130000.000000,69.069779,13.206974,15281,0 days 00:05:00,98.661443


In [13]:
# resetting the index
tracks_closer_AMF1_index_reset = tracks_closer_AMF1.reset_index(drop=True)
tracks_closer_AMF1_index_reset.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,453,121.000000,324.705104,2,0,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.181000e+06,241705.104259,70.248601,16.323888,5,0 days 00:00:00,125.680339
1,1,581,124.000000,327.838844,2,5,374,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.184000e+06,244838.843751,70.218311,16.396476,5,0 days 00:05:00,122.977127
2,2,716,127.000000,331.000000,1,10,780,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.187000e+06,248000.000000,70.187965,16.469553,5,0 days 00:10:00,120.373766
3,3,578,128.746166,333.673351,6,5,1063,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.188746e+06,250673.350542,70.169425,16.533521,5,0 days 00:15:00,119.040644
4,4,611,133.000000,334.000000,1,5,1406,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.193000e+06,251000.000000,70.130630,16.529381,5,0 days 00:20:00,114.859159


# Algorithm to calculate the corresponding reflectivity values (calculate for the max and mean dBZ) and the area

In [14]:
# unique_cell_ids = track_close_to_AMF1['cell'].values # getting the cells_tracked
# len(unique_cell_ids)

unique_cell_ids = np.unique(track_close_to_AMF1['cell'].values) # getting the cells_tracked
len(unique_cell_ids)

7121

In [15]:
%%time
# all masked cell in iris cube
masked_tracked_cell = []
for i, unique_value in enumerate(unique_cell_ids):
    # get masked cell based on cell id
    get_mask_cell = tobac.utils.mask_cell(mask=Mask_ERF, cell=unique_value, track=track, masked=False) 
    masked_tracked_cell.append(get_mask_cell)

CPU times: user 18.5 s, sys: 153 ms, total: 18.6 s
Wall time: 19.1 s


In [16]:
%%time
# Convert the segmented/mask cells from iris cube to xrray
masked_values_to_xarray = []
for i, items in enumerate(masked_tracked_cell):
    mask_convert_xr = xr.DataArray.from_iris(masked_tracked_cell[i]) # convert iris cube to xarray data array
    masked_values_to_xarray.append(mask_convert_xr) # append the results to list
    # print(mask_convert_xr)

CPU times: user 1min 3s, sys: 17.6 s, total: 1min 21s
Wall time: 1min 24s


In [17]:
%%time
# Getting the original masked reflectivity values
slice_cell_times = []
for i, cell_id in enumerate(np.unique(tracks_closer_AMF1['cell'])):
    cell_ids = tracks_closer_AMF1_index_reset[tracks_closer_AMF1_index_reset['cell'] == cell_id] 
    first_time = cell_ids['timestr'][::].values[0] # get first time (start time)
    last_time = cell_ids['timestr'][::-1].values[0]  # get the last time value (end time)
    # slice values based on time
    slice_cell_time = masked_values_to_xarray[i].sel(time=slice(first_time, last_time))
    slice_cell_times.append(slice_cell_time)
    # print(slice_cell_time)

CPU times: user 10.2 s, sys: 75.9 ms, total: 10.3 s
Wall time: 10.5 s


In [18]:
%%time
equivalent_reflectivity_factors = 10**((equivalent_reflectivity_factor/10))
# equivalent_reflectivity_factors = 10*np.log10(Z)

CPU times: user 4.04 s, sys: 252 ms, total: 4.3 s
Wall time: 4.4 s


## algorithm to extract the speed, area, maximum, minimum and mean reflectivity values based on cell_id and specific times

In [19]:
%%time
min_reflectivities = []; mean_reflectivities = []; max_reflectivities = []; areas = [];
for i, cell_id in enumerate(np.unique(tracks_closer_AMF1['cell'])):
    #################################
    import warnings
    warnings.filterwarnings('ignore')
    #################################
    cell_ids = tracks_closer_AMF1_index_reset[tracks_closer_AMF1_index_reset['cell'] == cell_id]  # dataframe based on cell_id
    first_time = cell_ids['timestr'][::].values[0] # get first time (start time)
    last_time = cell_ids['timestr'][::-1].values[0]  # get the last time value (end time)
    
#     # # # # Calculating area
    # area = tobac.analysis.calculate_area(track, Mask_ERF)
#     # area = tobac.analysis.calculate_area(cell_ids, masked_tracked_cell[i], method_area=None)
#     area = np.absolute(area['area']/1000000) # area in kilometers
#     areas.append(area)
    
    # area1 = np.divide(area['area'],1000000)
    # area['area($km^2$)'] = area1


    # reflectivity_masked based on times
    reflectivity_masked = equivalent_reflectivity_factors.sel(time=slice(first_time, last_time))
    
    # LOOPING THROUGH THE AND GETTING THE VALUES
    for index in (range(len(reflectivity_masked))):
        mean_ref = reflectivity_masked[index].mean().item() # get mean values of reflectivity # in dBZ values
        # # Convert from dBZ to Z values
        # mean_ref_Z = 10**(mean_ref / 10)
        # # Convert it back to dBZ units
        # mean_ref_dBZ = 10*np.log10(mean_ref_Z)
        #####################################
        max_ref = reflectivity_masked[index].max().item() # get max values of reflectivity
        
        mean_reflectivities.append(mean_ref)
        max_reflectivities.append(max_ref)
        
# concatenate the area values to one series
# area1 = pd.concat(areas)

# calculate velocity
# velocity = tobac.analysis.calculate_velocity(tracks_closer_AMF1_index_reset)

CPU times: user 1min 28s, sys: 307 ms, total: 1min 29s
Wall time: 1min 31s


In [20]:
# converting the mean Z unit back to dBZ
mean_reflectivities1 = 10*np.log10(mean_reflectivities)

# converting the max Z unit back to dBZ
max_reflectivities1 = 10*np.log10(max_reflectivities)

In [21]:
tracks_closer_AMF1_index_reset

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,453,121.000000,324.705104,2,0,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.181000e+06,241705.104259,70.248601,16.323888,5,0 days 00:00:00,125.680339
1,1,581,124.000000,327.838844,2,5,374,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.184000e+06,244838.843751,70.218311,16.396476,5,0 days 00:05:00,122.977127
2,2,716,127.000000,331.000000,1,10,780,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.187000e+06,248000.000000,70.187965,16.469553,5,0 days 00:10:00,120.373766
3,3,578,128.746166,333.673351,6,5,1063,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.188746e+06,250673.350542,70.169425,16.533521,5,0 days 00:15:00,119.040644
4,4,611,133.000000,334.000000,1,5,1406,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.193000e+06,251000.000000,70.130630,16.529381,5,0 days 00:20:00,114.859159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55639,299,476,293.326516,390.199380,15,15,66254,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.353327e+06,307199.379687,68.618348,17.437243,15279,0 days 00:05:00,91.303475
55640,300,539,295.648140,392.416781,10,15,66361,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.355648e+06,309416.781206,68.594721,17.483020,15279,0 days 00:10:00,94.427981
55641,298,555,260.000000,211.000000,1,25,66190,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.320000e+06,128000.000000,69.072189,13.157945,15281,0 days 00:00:00,100.574806
55642,299,531,260.153314,213.000000,3,25,66288,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.320153e+06,130000.000000,69.069779,13.206974,15281,0 days 00:05:00,98.661443


In [22]:
# append max min and mean reflectivity values based on each cell id and times to the tracks_closer_AMF1_index_reset dataframe
# tracks_closer_AMF1_index_reset['speed'] = velocity
# tracks_closer_AMF1_index_reset['area1'] = area1
tracks_closer_AMF1_index_reset['mean dBZ'] = pd.Series(mean_reflectivities1)
tracks_closer_AMF1_index_reset['max dBZ'] = pd.Series(max_reflectivities1)

In [23]:
area = tobac.calculate_area(tracks_closer_AMF1_index_reset, Mask_ERF, method_area=None)
# area1 = (c['area']/1000000)
area1 = np.divide(area['area'],1000000)
area['area($km^2$)'] = area1

In [24]:
final_tracks_thresh_all = tracks_closer_AMF1_index_reset.drop(columns=['area'])
final_tracks_thresh_all

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,453,121.000000,324.705104,2,0,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.181000e+06,241705.104259,70.248601,16.323888,5,0 days 00:00:00,125.680339,13.245867,34.567482,132.310998
1,1,581,124.000000,327.838844,2,5,374,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.184000e+06,244838.843751,70.218311,16.396476,5,0 days 00:05:00,122.977127,12.974534,35.577388,158.174980
2,2,716,127.000000,331.000000,1,10,780,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.187000e+06,248000.000000,70.187965,16.469553,5,0 days 00:10:00,120.373766,12.780750,34.063904,272.576605
3,3,578,128.746166,333.673351,6,5,1063,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.188746e+06,250673.350542,70.169425,16.533521,5,0 days 00:15:00,119.040644,12.708913,36.584485,247.709497
4,4,611,133.000000,334.000000,1,5,1406,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.193000e+06,251000.000000,70.130630,16.529381,5,0 days 00:20:00,114.859159,12.845599,38.092506,94.508120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55639,299,476,293.326516,390.199380,15,15,66254,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.353327e+06,307199.379687,68.618348,17.437243,15279,0 days 00:05:00,91.303475,15.240493,41.116443,117.421341
55640,300,539,295.648140,392.416781,10,15,66361,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.355648e+06,309416.781206,68.594721,17.483020,15279,0 days 00:10:00,94.427981,14.702386,40.106802,115.430654
55641,298,555,260.000000,211.000000,1,25,66190,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.320000e+06,128000.000000,69.072189,13.157945,15281,0 days 00:00:00,100.574806,15.301029,39.603226,100.492397
55642,299,531,260.153314,213.000000,3,25,66288,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.320153e+06,130000.000000,69.069779,13.206974,15281,0 days 00:05:00,98.661443,15.240493,41.116443,106.461253


In [25]:
#reset cell_id starting from 1
final_tracks_thresh_all['cloud_id'] = final_tracks_thresh_all.groupby('cell').ngroup() + 1
final_tracks_thresh_all

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),cloud_id
0,0,453,121.000000,324.705104,2,0,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.181000e+06,241705.104259,70.248601,16.323888,5,0 days 00:00:00,125.680339,13.245867,34.567482,132.310998,1
1,1,581,124.000000,327.838844,2,5,374,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.184000e+06,244838.843751,70.218311,16.396476,5,0 days 00:05:00,122.977127,12.974534,35.577388,158.174980,1
2,2,716,127.000000,331.000000,1,10,780,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.187000e+06,248000.000000,70.187965,16.469553,5,0 days 00:10:00,120.373766,12.780750,34.063904,272.576605,1
3,3,578,128.746166,333.673351,6,5,1063,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.188746e+06,250673.350542,70.169425,16.533521,5,0 days 00:15:00,119.040644,12.708913,36.584485,247.709497,1
4,4,611,133.000000,334.000000,1,5,1406,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.193000e+06,251000.000000,70.130630,16.529381,5,0 days 00:20:00,114.859159,12.845599,38.092506,94.508120,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55639,299,476,293.326516,390.199380,15,15,66254,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.353327e+06,307199.379687,68.618348,17.437243,15279,0 days 00:05:00,91.303475,15.240493,41.116443,117.421341,7120
55640,300,539,295.648140,392.416781,10,15,66361,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.355648e+06,309416.781206,68.594721,17.483020,15279,0 days 00:10:00,94.427981,14.702386,40.106802,115.430654,7120
55641,298,555,260.000000,211.000000,1,25,66190,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.320000e+06,128000.000000,69.072189,13.157945,15281,0 days 00:00:00,100.574806,15.301029,39.603226,100.492397,7121
55642,299,531,260.153314,213.000000,3,25,66288,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.320153e+06,130000.000000,69.069779,13.206974,15281,0 days 00:05:00,98.661443,15.240493,41.116443,106.461253,7121


In [26]:
final_tracks_thresh_all = final_tracks_thresh_all.sort_values(by=['timestr', 'time'])
final_tracks_thresh_all

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),cloud_id
0,0,453,121.000000,324.705104,2,0,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.181000e+06,241705.104259,70.248601,16.323888,5,0 days 00:00:00,125.680339,13.245867,34.567482,132.310998,1
5,0,456,135.877490,273.231217,7,0,6,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.195877e+06,190231.216954,70.159887,14.951233,6,0 days 00:00:00,116.846876,13.245867,34.567482,331.269006,2
9,0,471,168.000000,286.000000,1,0,10,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.228000e+06,203000.000000,69.858376,15.206022,10,0 days 00:00:00,81.936672,13.245867,34.567482,65.658469,3
15,0,481,203.473824,247.478730,4,0,13,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.263474e+06,164478.729588,69.565035,14.156179,13,0 days 00:00:00,76.275220,13.245867,34.567482,327.324894,4
19,0,482,204.000000,286.176995,2,0,14,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.264000e+06,203176.994856,69.531731,15.128128,14,0 days 00:00:00,48.610059,13.245867,34.567482,171.124975,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55631,300,555,178.000000,358.012383,2,20,66372,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.238000e+06,275012.383147,69.698076,17.005561,15276,0 days 00:10:00,80.687235,14.702386,40.106802,208.928144,7117
55634,300,575,285.000000,350.000000,1,20,66383,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.345000e+06,267000.000000,68.738228,16.495682,15277,0 days 00:10:00,55.407771,14.702386,40.106802,214.931734,7118
55637,300,576,286.727785,374.604901,6,20,66384,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.346728e+06,291604.900654,68.696045,17.083278,15278,0 days 00:10:00,74.819683,14.702386,40.106802,494.549355,7119
55640,300,539,295.648140,392.416781,10,15,66361,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.355648e+06,309416.781206,68.594721,17.483020,15279,0 days 00:10:00,94.427981,14.702386,40.106802,115.430654,7120


In [27]:
# all tracks
# final_tracks_thresh_10.to_csv('../cells-closer-to-AMF1/cells-closer-AMF1_allkm-KAZR-thres-all.csv', index=False)
final_tracks_thresh_all.to_csv('../saved-files/threshold-all/cells-allkm-KAZR-thres-all-all.csv', index=False)

# END

In [ ]:
# tracks = pd.read_csv('../saved-files/threshold-all/cells-allkm-KAZR-thres-all-all.csv')
# tracks.head()

In [ ]:
# len(tracks)

# Identifying Cells that were found in KAZR with C-band

### testing for cell all cells

In [ ]:
def all_kazr_cells(start_time:str, end_time:str, CAO_date:str, folder_name:str, gif_name:str):
    """Generate a GIF animation of all cells within a given time range.

    Parameters:
        start_time (str): The start time of the animation in the format '"%H:%M:%S %d %b %Y".'.
        end_time (str): The end time of the animation in the format '"%H:%M:%S %d %b %Y".'.
        CAO_date(str): The times of CAO.
        folder_name (str): The name of the folder containing the cell data.
        name_of_gif (str): The desired name for the generated GIF file.

    Returns:
        animation of cell"""
    
        
    # read track files
    tracks = pd.read_csv('../saved-files/threshold-all/cells-allkm-KAZR-thres-all-all.csv')
    # converting time object to datetime format
    tracks['time'] = pd.to_datetime(tracks['time'])
    tracks['timestr'] = pd.to_datetime(tracks['timestr'])
#     
    # mask the start and end time
    mask_date = (tracks['timestr'] >= start_time) & (tracks['timestr'] <= end_time)
    cell1 = tracks[(mask_date)].reset_index(drop=True)
    
    # get unique values
    idx = np.unique(cell1['cell'].values)
    
    # get all masked cell and convert to xarray
    convert_alls = []
    for ii, cell_id in enumerate(idx):
        # mask in iris cube
        mask_cell_all = tobac.utils.mask_cell(Mask_ERF, cell_id, tracks)
        # convert to xarray dataarray
        convert_all = xr.DataArray.from_iris(mask_cell_all)
        convert_alls.append(convert_all)
        
    
    # plots
    loop_over = cell1['frame'].values[:]
    cells = cell1['cell'].values[:]
    for ind, i in enumerate(loop_over):
        latA = 69.141281 #latitude of COMBLE site
        lonA = 15.684166-1 #longitude of COMBLE site -1
        xm, ym = 6,2.75 # extent from center point in lon and lat

        fig, ax = plt.subplots(1,1,figsize=(10,9),subplot_kw={'projection': ccrs.Orthographic(lonA,latA)})
        # xm, ym = 6,2.75 # extent from center point in lon and lat

        # extent = (lonA-xm, lonA+xm, latA-ym, latA+ym) # define extent map
        extent = (lonA-xm+3.3, lonA+xm-3.7, latA-ym+2, latA+ym-1.5) # define extent map
        ax.set_extent(extent) # set extent of map
        ax.coastlines(resolution='10m', color='black') # plot coastlines with high resolution: 10m
        ax.set_facecolor('white')
        ax.add_feature(cfeature.LAND, linewidth=1.5, alpha=0.5)

        # add grid lines
        gl = ax.gridlines(x_inline=False, alpha=0.3, color='white'); gl.bottom_labels=True; gl.left_labels=True
        gl.xformatter = LONGITUDE_FORMATTER; gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'rotation': 45}; gl.ylabel_style = {'color': 'black'}


        #############################
        # loop through cell-id
        for cell_idx in range(0, len(idx)):
            # change time to moth, day, year, hours and minutes
            dt = masked_values_to_xarray[cell_idx]['time'].dt.strftime('%m-%d-%Y %H:%M').values  ### array 

             # fill in the place where there is cloud with 1 and place with no cloud to be zero
            cloud = np.asarray(convert_alls[cell_idx][i], dtype='float')
            invalid_cloud = np.greater(cloud, 0)
            cloud[invalid_cloud] = 1
            invalid_cloud1 = np.less(cloud, 1)
            cloud[invalid_cloud1] = np.nan


            cs=ax.pcolormesh(convert_alls[cell_idx]['lon'],
                        convert_alls[cell_idx]['lat'], 
                        # cloud*equivalent_reflectivity_factor.values[i],
                        cloud*equivalent_reflectivity_factor.values[i],
                        cmap='jet',
                        transform=ccrs.PlateCarree(),
                        vmin=-30,
                        vmax=30)


             # display each id on cell
            frame = cell1[cell1['frame']==i]
            longi = frame['longitude'].values[:]
            lati = frame['latitude'].values[:]
            areas = frame['area($km^2$)'].values[:]
            cell_id = frame['cell'].values[:]
            cloud_id = frame['cloud_id'].values[:]

            #     # check if area has value of nan
            if not math.isnan(frame['area($km^2$)'].values[0]):
                # for lo, la, ce, are in zip(all_long, all_lat, all_idx, all_area):
                for lon, lat, cell_ids, area in zip(longi, lati, cell_id, areas):
                    fontdict={'fontweight': 'extra bold'}
                    ax.text(lon, lat, cell_ids, transform=ccrs.PlateCarree(), fontsize=10, color='black', fontdict=fontdict)


        ax.plot(lonA+1,latA, color='red', marker='*', markersize = 7.5,transform=ccrs.PlateCarree()) # plot red star at location of Andenes
        ax.set_title(f"Nordic Radar Mosaic: {dt[i]} UTC", fontweight='bold', fontsize=16, pad=11)  # title of figure


        # Add a colorbar axis at the right of the graph
        cbar_ax = fig.add_axes([0.85, 0.11, 0.047, 0.77]) #lbwh



        cbar=fig.colorbar(cs, cax=cbar_ax, orientation='vertical')
        cbar.set_label("Reflectivity Factor [dBZ]", fontsize = 15)
        
        # Specify the directory and file name
        directory = f'./Figures/{CAO_date}/{folder_name}/'
        file_name = f'{dt[i]}.png'
        
        # Create the directory if it doesn't exist
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(f"Directory '{directory}' created successfully.")

            

    
        # save figure
        # fig.savefig(os.path.join(directory, file_name), dpi=300)
        # plt.close()
        
        dirr = sorted(os.listdir(directory))
        if f'{dt[i]}.png' in dirr:
            # print(f'{dt[i]}.png already exist')
            pass
            plt.close()
        else:
            # fig.savefig(os.path.join(directory, file_name), dpi=100)
            # print(f"Figures in {file_name} executed!")
            plt.close()
    print('\U0001f600\U0001f600\U0001f600\U0001f600EXECUTION DONE!!!\U0001f600\U0001f600\U0001f600\U0001f600')
    # gc.collect()

        
        
    # build gif
    
    imgs = sorted(glob.glob(f"{directory}/*png"))
    with imageio.get_writer(f"{directory}/{gif_name}.gif", mode='I', duration=0.7) as writer:
        for filename in imgs:
            image = imageio.v2.imread(filename)
            writer.append_data(image)
    print(f"Figures in {folder_name} executed!")

In [ ]:
# %%time
one = all_kazr_cells(start_time='08:12:28 13 Mar 2020', end_time='09:06:36 13 Mar 2020', 
                CAO_date='try',
                folder_name='cell1', 
                gif_name='cell-id1')
one

In [45]:
# Data containing the ID, start date, and end date
kazr_cell_time = [
    (1, "08:12:28 13 Mar 2020", "09:06:36 13 Mar 2020"),
    (2, "09:12:48 13 Mar 2020", "09:58:16 13 Mar 2020"),
    (3, "10:00:12 13 Mar 2020", "10:11:56 13 Mar 2020"),
    (4, "10:14:32 13 Mar 2020", "10:29:00 13 Mar 2020"),
    (5, "10:22:40 13 Mar 2020", "10:46:56 13 Mar 2020"),
    (6, "10:42:32 13 Mar 2020", "10:59:04 13 Mar 2020"),
    (7, "11:00:32 13 Mar 2020", "11:32:28 13 Mar 2020"),
    (8, "11:30:44 13 Mar 2020", "11:51:52 13 Mar 2020"),
    (9, "11:48:08 13 Mar 2020", "12:09:04 13 Mar 2020"),
    (10, "12:03:08 13 Mar 2020", "12:38:04 13 Mar 2020"),
    (11, "12:32:48 13 Mar 2020", "13:14:32 13 Mar 2020"),
    (12, "13:53:00 13 Mar 2020", "14:05:16 13 Mar 2020"),
    (13, "14:03:28 13 Mar 2020", "14:31:20 13 Mar 2020"),
    (14, "15:09:36 13 Mar 2020", "15:45:20 13 Mar 2020"),
    (15, "15:48:00 13 Mar 2020", "16:12:44 13 Mar 2020"),
    (16, "16:12:00 13 Mar 2020", "16:33:44 13 Mar 2020"),
    (17, "16:29:40 13 Mar 2020", "16:38:20 13 Mar 2020"),
    (18, "16:44:24 13 Mar 2020", "17:13:32 13 Mar 2020"),
    (19, "17:12:56 13 Mar 2020", "17:45:48 13 Mar 2020"),
    (20, "18:10:04 13 Mar 2020", "18:28:04 13 Mar 2020"),
    (21, "18:27:28 13 Mar 2020", "18:50:04 13 Mar 2020"),
    (22, "19:15:32 13 Mar 2020", "19:54:00 13 Mar 2020"),
    (23, "19:47:12 13 Mar 2020", "20:15:52 13 Mar 2020"),
    (24, "20:14:12 13 Mar 2020", "20:48:12 13 Mar 2020"),
    (25, "20:48:36 13 Mar 2020", "21:21:40 13 Mar 2020"),
    (26, "21:27:40 13 Mar 2020", "21:50:48 13 Mar 2020"),
    (27, "21:48:20 13 Mar 2020", "22:13:00 13 Mar 2020"),
    (28, "22:28:24 13 Mar 2020", "23:00:00 13 Mar 2020"),
    (29, "23:38:56 13 Mar 2020", "00:16:28 14 Mar 2020"),
    (30, "00:13:24 14 Mar 2020", "00:30:44 14 Mar 2020"),
    (31, "01:06:40 14 Mar 2020", "01:24:08 14 Mar 2020"),
    (32, "02:19:20 14 Mar 2020", "02:34:36 14 Mar 2020"),
    (33, "02:43:24 14 Mar 2020", "03:12:44 14 Mar 2020"),
    (34, "03:17:16 14 Mar 2020", "03:41:20 14 Mar 2020"),
    (35, "03:57:16 14 Mar 2020", "04:31:56 14 Mar 2020"),
    (36, "04:27:40 14 Mar 2020", "04:50:20 14 Mar 2020"),
    (37, "04:49:56 14 Mar 2020", "05:19:48 14 Mar 2020")
]

In [46]:
for i in kazr_cell_time:
    # extract variables
    kazr_id, kazr_start_time, kazr_end_time = int(i[0]), i[1], i[2]
    ## display images and animations for all track-cells for March 13 and March 14
    all_kazr_cells(start_time=kazr_start_time, 
                   end_time=kazr_end_time,
                   CAO_date="Mar-13 & Mar-14",
                  folder_name=f'ID-{kazr_id}', 
                 gif_name=f'ID-{kazr_id}')

Directory './figures/Mar-13 & Mar-14/ID-1/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-1 executed!
Directory './figures/Mar-13 & Mar-14/ID-2/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-2 executed!
Directory './figures/Mar-13 & Mar-14/ID-3/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-3 executed!
Directory './figures/Mar-13 & Mar-14/ID-4/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-4 executed!
Directory './figures/Mar-13 & Mar-14/ID-5/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-5 executed!
Directory './figures/Mar-13 & Mar-14/ID-6/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-6 executed!
Directory './figures/Mar-13 & Mar-14/ID-7/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-7 executed!
Directory './figures/Mar-13 & Mar-14/ID-8/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-8 executed!
Directory './figures/Mar-13 & Mar-14/ID-9/' created succ

# END 